In [1]:
# Импорт необходимых библиотек
from selenium import webdriver
from bs4 import BeautifulSoup as BS 
import time
import pandas as pd

In [2]:
# Подключение драйвера для парсинга
links = list()
url = "https://www.cian.ru/kupit-dom/"
driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')
driver.get(url)

<ipython-input-2-a1d27bf7d722>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


In [3]:
# Получаем ссылки на все страницы домов с первой страницы сайта
html = BS(driver.page_source, 'html.parser')
items = html.find_all('a', href=True, class_="_93444fe79c--link--39cNw")
for i in items:
    links.append(i['href'])
# 54 страницы на сайте

In [4]:
links = list()
# Получаем ссылки на все страницы домов с оставшихся страниц сайта
for page in range(2, 55):
    url = "https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&object_type%5B0%5D=1&offer_type=suburban&p="+str(page)+"&region=1"
    driver.get(url)
    html = BS(driver.page_source, 'html.parser')
    items = html.find_all('a', href=True, class_="_93444fe79c--link--39cNw")
    for i in items:
        links.append(i['href'])

In [5]:
# Разделяем все полученные ссылки на 10 групп
import more_itertools as mit

links = [list(c) for c in mit.divide(10, links)]

In [6]:
# Создаем dataframe для записи полученных данных
columns = ['type', 'price', 'priсePerEarthsquare', 'houseSquare', 'earthSquare', 'floors', 'place', 'URL']
df = pd.DataFrame(columns=columns)
df

,type,price,prisePerEarthsquare,houseSquare,earthSquare,floors,place,URL


In [9]:
# Создаем функцию для парсинга информации о домах
def parse(mass):
    count = 1
    statistics = list()

    i = mass

    driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')

    for link in i:
        parameters = list()
    
        driver.get(link)
        html = BS(driver.page_source, 'html.parser')
    
        try:
            p = html.find('h1', class_="a10a3f92e9--title--2Widg").text
        except:
            time.sleep(60)
            driver.get(link)
            html = BS(driver.page_source, 'html.parser')
            p = html.find('h1', class_="a10a3f92e9--title--2Widg").text
        if "дом" in p or 'Дом' in p:
            parameters.append('house')
        elif "коттедж" in p or 'Коттедж' in p:
            parameters.append('cottage')
        
        p = html.find('span', itemprop='price')
        parameters.append("".join(p.text.split('\xa0')[:-1]))
    
        p = html.find('div', class_="a10a3f92e9--price_per_meter--hKPtN a10a3f92e9--price_per_meter--residential--1mFDW")
        parameters.append(''.join(p.text[:-5].split()))
    
        p = html.find('div', class_="a10a3f92e9--info-block--3Va89")
        d = list()
        for i in p:
            a = i.find_all('div')
            for j in a:
                d.append(j.text)
                
        for i in range(1, len(d)//2 + 1):
            del d[i]
            
        if len(d) == 4:
            del d[2]
            
        if len(d) == 5:
            del d[2]
            del d[3]
            
        if len(d) == 2:
            d.append(None)
            
        d[0] = d[0][:-3]
        
        for j in d:
            parameters.append(j)
            
        parameters[4] = parameters[4][:-5]
        
        p = html.find('a', class_="a10a3f92e9--link--1t8n1 a10a3f92e9--highway_link--1jVab")
        if p == None:
            parameters.append(None)
        else:
            parameters.append(p.text)
        
        parameters.append(link)
        
        if parameters[-3] != None and len(parameters[-3]) > 1:
            parameters[-3] = None
            
        if " " in parameters[3]:
            parameters[3] = ''.join(parameters[3].split())
        if "," in parameters[3]:
            parameters[3] = parameters[3].replace(',' , '.')
            
        print(parameters, count)
        
        count += 1
        
        statistics.append(parameters)
        
    return list(statistics)

In [10]:
list1 = parse(links[0]) # Первый запуск функции

<ipython-input-9-b26b98531458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


['cottage', '78000000', '197468', '395', '22.3', '4', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/248884179/'] 1
['cottage', '88500000', '305172', '290', '17.58', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/256413509/'] 2
['cottage', '97007894', '195976', '495', '20.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/238388603/'] 3
['house', '65655917', '76755', '855.4', '20.1', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/249971632/'] 4
['cottage', '33500000', '97101', '345', '10.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264321673/'] 5
['cottage', '42800000', '58231', '735', '24.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264886768/'] 6
['cottage', '72000000', '128571', '560', '30.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262526633/'] 7
['cottage', '240708120', '240708', '1000', '95.57', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/238387572/'] 8
['house', '189000

['house', '15700000', '130833', '120', '8.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265269186/'] 68
['house', '18500000', '92500', '200', '9.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/263451372/'] 69
['cottage', '19000000', '74510', '255', '10.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/263883858/'] 70
['cottage', '22000000', '81481', '270', '10.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264970888/'] 71
['house', '25000000', '100000', '250', '10.6', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264024135/'] 72
['house', '25700000', '128500', '200', '17.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262183074/'] 73
['house', '27900000', '91776', '304', '9.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/218387719/'] 74
['house', '34000000', '80000', '425', '16.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/263904849/'] 75
['house', '35000000', '1166

['house', '120000000', '171429', '700', '95.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/158158079/'] 135
['house', '122158000', '142044', '860', '21.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262988298/'] 136
['cottage', '179440000', '313159', '573', '15.0', '4', 'Куркинское шоссе', 'https://www.cian.ru/sale/suburban/262080513/'] 137
['house', '3700000', '41111', '90', '5.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/256001408/'] 138
['house', '3800000', '54286', '70', '5.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/261776974/'] 139
['house', '10799000', '95145', '113.5', '3.6', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/265337178/'] 140
['cottage', '12000000', '48000', '250', '8.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/265298501/'] 141
['house', '12550000', '56430', '222.4', '10.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/259496521/'] 142
['cottage', '1300

In [11]:
list2 = parse(links[1]) # Второй запуск функции

<ipython-input-9-b26b98531458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


['cottage', '24000000', '80000', '300', '16.0', '2', 'Симферопольское шоссе', 'https://www.cian.ru/sale/suburban/260802510/'] 1
['cottage', '25000000', '89190', '280.3', '15.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/253645124/'] 2
['house', '28730000', '51645', '556.3', '12.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/255628793/'] 3
['house', '33600000', '81553', '412', '12.5', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/265330541/'] 4
['cottage', '34900000', '116333', '300', '8.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/261777243/'] 5
['house', '37500000', '83333', '450', '10.0', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/253215446/'] 6
['house', '38000000', '118750', '320', '8.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264479396/'] 7
['house', '40000000', '84211', '475', '12.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264086102/'] 8
['house', '40000000', '7692

['house', '32000000', '61069', '524', '10.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/152150048/'] 69
['house', '32000000', '91429', '350', '20.0', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264884304/'] 70
['house', '32000000', '88203', '362.8', '15.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/255628794/'] 71
['house', '32500000', '104839', '310', '16.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262166023/'] 72
['house', '33000000', '92958', '355', '11.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/229448809/'] 73
['house', '34200000', '114000', '300', '8.0', None, 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/262715475/'] 74
['house', '34500000', '138000', '250', '20.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264174860/'] 75
['house', '34900000', '74255', '470', '12.0', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264352314/'] 76
['house', '34900000', '16778

['house', '58000000', '290000', '200', '4.55', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/259195756/'] 137
['house', '58348309', '162530', '359', '15.0', '2', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/265368592/'] 138
['house', '59000000', '168571', '350', '39.0', '2', 'Каширское шоссе', 'https://www.cian.ru/sale/suburban/256179430/'] 139
['house', '59000000', '56950', '1036', '30.0', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/242285778/'] 140
['cottage', '59900000', '149750', '400', '14.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265224650/'] 141
['house', '60000000', '169492', '354', '20.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/261600692/'] 142
['house', '60000000', '98847', '607', '20.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/254812802/'] 143
['house', '59900000', '172822', '346.6', '45.0', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264638875/'] 144
['house', '6000000

In [12]:
list3 = parse(links[2]) # Третий запуск функции

<ipython-input-9-b26b98531458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


['house', '60053420', '170123', '353', '14.7', '2', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/265368601/'] 1
['cottage', '61000000', '174286', '350', '15.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/239293134/'] 2
['house', '62000000', '134490', '461', '22.5', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/263565797/'] 3
['house', '62475000', '97465', '641', '31.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/253399068/'] 4
['house', '63000000', '96923', '650', '11.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265372618/'] 5
['cottage', '63000000', '96923', '650', '10.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/219280670/'] 6
['cottage', '65000000', '130000', '500', '35.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/251817145/'] 7
['house', '65000000', '154762', '420', '15.0', '4', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/256128284/'] 8
['house', '66070208', '191947'

['house', '150000000', '172414', '870', '30.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264853963/'] 69
['house', '154494055', '190733', '810', '30.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/232095299/'] 70
['house', '179644250', '178573', '1006', '34.0', '4', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265214557/'] 71
['cottage', '180000000', '360000', '500', '20.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/218872284/'] 72
['house', '185000000', '212644', '870', '20.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/258949921/'] 73
['cottage', '185000000', '430233', '430', '13.0', '3', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/255550927/'] 74
['house', '197608674', '171834', '1150', '30.0', '2', 'Сколковское шоссе', 'https://www.cian.ru/sale/suburban/256430830/'] 75
['house', '199000000', '279887', '711', '30.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265195405/'] 76
['house',

['house', '46000000', '104545', '440', '8.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/246165960/'] 136
['cottage', '46500000', '104494', '445', '15.5', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/261073081/'] 137
['cottage', '49500000', '151840', '326', '12.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264722786/'] 138
['house', '49900000', '123210', '405', '41.0', '4', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/258245849/'] 139
['house', '51000000', '143662', '355', '10.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/259853084/'] 140
['cottage', '52000000', '117914', '441', '10.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/251346483/'] 141
['cottage', '52300000', '174333', '300', '20.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/258107763/'] 142
['house', '54000000', '106593', '506.6', '14.5', '4', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/223993199/'] 143
['house

In [13]:
list4 = parse(links[3]) # Четвертый запуск функции

<ipython-input-9-b26b98531458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


['house', '60000000', '148148', '405', '20.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/226477057/'] 1
['house', '60000000', '92308', '650', '10.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/223993466/'] 2
['house', '65000000', '212905', '305.3', '8.56', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/257945265/'] 3
['house', '65000000', '212905', '305.3', '8.56', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/257945265/'] 4
['house', '65000000', '85752', '758', '13.22', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/239507250/'] 5
['house', '66900000', '152740', '438', '35.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262921256/'] 6
['house', '67500000', '192857', '350', '15.0', '4', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/238889951/'] 7
['house', '67000000', '105512', '635', '16.5', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/238122932/'] 8
['cottage', '67546237', '1

['cottage', '17500000', '96154', '182', '10.0', '2', 'Симферопольское шоссе', 'https://www.cian.ru/sale/suburban/256542480/'] 69
['cottage', '17580000', '38217', '460', '6.0', '3', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/260483024/'] 70
['house', '19200000', '81356', '236', '9.6', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264815688/'] 71
['house', '20500000', '113889', '180', '8.5', '2', 'Дмитровское шоссе', 'https://www.cian.ru/sale/suburban/265219095/'] 72
['cottage', '23000000', '88462', '260', '7.5', '2', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/260485951/'] 73
['house', '25000000', '80645', '310', '10.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/191794702/'] 74
['house', '27800000', '73158', '380', '8.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262152753/'] 75
['cottage', '28535000', '115526', '247', '10.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/259183303/'] 76
['house', '29500000',

['house', '8750000', '63177', '138.5', '6.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265249486/'] 136
['house', '4950000', '48529', '102', '7.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/265108424/'] 137
['cottage', '12000000', '80000', '150', '5.2', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/264632532/'] 138
['cottage', '15000000', '100000', '150', '8.09', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/260921640/'] 139
['house', '21000000', '95455', '220', '10.0', '3', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/226026908/'] 140
['cottage', '53000000', '103922', '510', '24.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/261214558/'] 141
['cottage', '68000000', '185286', '367', '15.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/258433105/'] 142
['house', '13450000', '46046', '292.1', '26.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/246689051/'] 143
['house', '55

In [14]:
list5 = parse(links[4]) # Пятый запуск функции

<ipython-input-9-b26b98531458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


['cottage', '12500000', '83333', '150', '7.5', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/265184478/'] 1
['house', '17500000', '116667', '150', '8.0', None, 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/260002626/'] 2
['house', '27000000', '42188', '640', '21.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/261259184/'] 3
['house', '28900000', '76053', '380', '20.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/226323307/'] 4
['house', '29500000', '58416', '505', '25.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/195762637/'] 5
['house', '32000000', '91429', '350', '10.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/261937017/'] 6
['house', '36000000', '180000', '200', '9.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/192946221/'] 7
['cottage', '37500000', '150000', '250', '15.0', None, 'Минское шоссе', 'https://www.cian.ru/sale/suburban/258949779/'] 8
['house', '54000000', '216000', '250

['house', '15000000', '78947', '190', '11.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/257538142/'] 70
['cottage', '15000000', '75000', '200', '9.0', '2', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/264857493/'] 71
['cottage', '15000000', '75000', '200', '9.0', '2', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/264826350/'] 72
['house', '15500000', '107639', '144', '11.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/264748133/'] 73
['cottage', '15500000', '55357', '280', '11.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/260766069/'] 74
['house', '17000000', '113333', '150', '20.0', '2', 'Новорижское шоссе', 'https://www.cian.ru/sale/suburban/265087167/'] 75
['cottage', '17000000', '88542', '192', '4.5', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/245912542/'] 76
['cottage', '17500000', '116667', '150', '8.0', None, 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/264451861/'] 77
['house', '17500000', 

['cottage', '40000000', '133333', '300', '6.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/180611635/'] 138
['house', '40000000', '263158', '152', '6.5', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/265267477/'] 139
['cottage', '41900000', '120749', '347', '8.93', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/259128865/'] 140
['cottage', '43000000', '122857', '350', '15.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/265240726/'] 141
['cottage', '45000000', '145161', '310', '12.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262953108/'] 142
['cottage', '45000000', '90000', '500', '8.16', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/255428007/'] 143
['cottage', '45000000', '180000', '250', '15.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/209026385/'] 144
['house', '45000000', '150000', '300', '15.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264948354/'] 145
['house',

In [16]:
list6 = parse(links[5]) # Шестой запуск функции

<ipython-input-9-b26b98531458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


['house', '47500000', '262431', '181', '4.74', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264242681/'] 1
['house', '48000000', '109091', '440', '13.4', '2', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/265380253/'] 2
['house', '49000000', '173759', '282', '4.8', '3', None, 'https://www.cian.ru/sale/suburban/259230224/'] 3
['house', '49500000', '141429', '350', '5.0', '2', 'Дмитровское шоссе', 'https://www.cian.ru/sale/suburban/258520154/'] 4
['cottage', '51700000', '94000', '550', '7.0', '2', None, 'https://www.cian.ru/sale/suburban/202242133/'] 5
['house', '52500000', '145833', '360', '12.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/252689729/'] 6
['cottage', '52499000', '131576', '399', '17.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/256400083/'] 7
['cottage', '53000000', '168254', '315', '6.31', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/258075437/'] 8
['cottage', '53000000', '94643', '560', '8.0', '3', None

['cottage', '39000000', '32500', '1200', '25.0', '4', 'Симферопольское шоссе', 'https://www.cian.ru/sale/suburban/264724154/'] 68
['house', '17964424', '56139', '320', '15.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/261053679/'] 69
['house', '2200000', '31429', '70', '6.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/243916900/'] 70
['house', '3500000', '50000', '70', '10.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264836370/'] 71
['house', '9700000', '57059', '170', '7.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264442575/'] 72
['cottage', '11850000', '59250', '200', '8.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/263040299/'] 73
['house', '17000000', '184783', '92', '12.0', '2', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/244451235/'] 74
['house', '26900000', '49177', '547', '25.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262640874/'] 75
['house', '41000000', '164000'

['house', '125000000', '335121', '373', '15.0', '3', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/264418539/'] 136
['cottage', '170000000', '113333', '1500', '100.0', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/227416767/'] 137
['house', '204794444', '215573', '950', '36.5', '3', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/226365727/'] 138
['house', '220000000', '362438', '607', '27.0', '4', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264146494/'] 139
['house', '240000000', '200000', '1200', '50.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262795209/'] 140
['house', '301802339', '111779', '2700', '50.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/262795213/'] 141
['house', '2155730999', '1437154', '1500', '160.0', None, 'Минское шоссе', 'https://www.cian.ru/sale/suburban/263280428/'] 142
['house', '1550000', '77500', '20', '6.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/260557767/'] 143
['house'

In [17]:
list7 = parse(links[6]) # Седбмой запуск функции

<ipython-input-9-b26b98531458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


['house', '2700000', '40909', '66', '5.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/215748402/'] 1
['house', '2990000', '59800', '50', '4.9', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265181078/'] 2
['house', '3000000', '52632', '57', '6.17', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/262350952/'] 3
['house', '3150000', '81818', '38.5', '5.3', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/263962650/'] 4
['house', '3200000', '53333', '60', '6.0', '2', 'Симферопольское шоссе', 'https://www.cian.ru/sale/suburban/262706223/'] 5
['house', '3300000', '47143', '70', '10.0', '1', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/259216353/'] 6
['house', '3300000', '55000', '60', '6.3', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/247516538/'] 7
['house', '3700000', '35171', '105.2', '6.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264917442/'] 8
['house', '3700000', '37000', '100', '5.0', '2', 'Ва

['house', '6970000', '55317', '126', '6.0', '2', 'Симферопольское шоссе', 'https://www.cian.ru/sale/suburban/261991707/'] 70
['house', '6950000', '115833', '60', '8.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/257646710/'] 71
['house', '7000000', '87500', '80', '7.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265088347/'] 72
['house', '7000000', '87500', '80', '10.0', '2', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/263121525/'] 73
['house', '7000000', '233333', '30', '9.0', '2', None, 'https://www.cian.ru/sale/suburban/264981008/'] 74
['house', '7000000', '304348', '23', '15.0', '1', 'Ленинградское шоссе', 'https://www.cian.ru/sale/suburban/262801606/'] 75
['house', '7345000', '63870', '115', '3.2', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/259929093/'] 76
['house', '7300000', '76842', '95', '6.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/263883653/'] 77
['house', '7300000', '146000', '50', '6.0', '2', '

['house', '10000000', '62500', '160', '12.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/236008675/'] 138
['house', '10199999', '223684', '45.6', '18.0', '1', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/260010979/'] 139
['house', '10150000', '61515', '165', '6.6', '2', 'Носовихинское шоссе', 'https://www.cian.ru/sale/suburban/234929524/'] 140
['house', '10300000', '103000', '100', '5.0', '2', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/262122169/'] 141
['cottage', '10500000', '30000', '350', '24.0', '4', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/257484301/'] 142
['house', '10500000', '28378', '370', '18.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/253178396/'] 143
['house', '10500000', '60345', '174', '6.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264035382/'] 144
['house', '10500000', '100000', '105', '7.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/263864436/'] 145
['cottage', '1050000

In [18]:
list8 = parse(links[7]) # Восьмой запуск функции

<ipython-input-9-b26b98531458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


['house', '10500000', '140000', '75', '14.0', '1', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/258379248/'] 1
['house', '10500000', '84000', '125', '9.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/264654584/'] 2
['cottage', '10490000', '38708', '271', '15.0', '3', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/237379923/'] 3
['house', '10650000', '158718', '67.1', '6.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/263158835/'] 4
['house', '10800000', '51429', '210', '12.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/261792100/'] 5
['house', '10800000', '180000', '60', '15.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/264816037/'] 6
['house', '10900000', '68125', '160', '16.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/259291645/'] 7
['house', '10950000', '109500', '100', '6.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/261396807/'] 8
['cottage', '10900000', '43600', '25

['house', '13890000', '63136', '220', '12.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/261917270/'] 69
['house', '13800000', '50182', '275', '12.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/241154917/'] 70
['cottage', '14000000', '56000', '250', '12.0', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264083734/'] 71
['house', '13990000', '46633', '300', '18.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/263103831/'] 72
['house', '13950000', '77500', '180', '8.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264738563/'] 73
['cottage', '14000000', '96552', '145', '5.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265032733/'] 74
['cottage', '14000000', '39326', '356', '8.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265115717/'] 75
['house', '14000000', '148936', '94', '23.0', '2', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/261426255/'] 76
['cottage', '14500000', '162

['house', '17000000', '89474', '190', '5.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/256633053/'] 137
['house', '17000000', '35052', '485', '10.0', '4', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/150934702/'] 138
['house', '17300000', '86500', '200', '10.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262551815/'] 139
['cottage', '17200000', '144538', '119', '4.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/261080013/'] 140
['cottage', '17500000', '81019', '216', '8.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/248758657/'] 141
['cottage', '17490000', '50843', '344', '15.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/258213059/'] 142
['house', '17500000', '49435', '354', '15.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262552711/'] 143
['house', '17490000', '50858', '343.9', '15.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/246853924/'] 144
['house', '175000

In [19]:
list9 = parse(links[8]) # Девятый запуск функции

<ipython-input-9-b26b98531458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


['cottage', '17490000', '53000', '330', '15.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/253519294/'] 1
['house', '17500000', '152174', '115', '15.0', '2', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/257321783/'] 2
['house', '17500000', '125000', '140', '6.8', '2', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/264535454/'] 3
['cottage', '18000000', '28571', '630', '14.7', '4', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/253437287/'] 4
['house', '18000000', '34951', '515', '20.0', '3', 'Варшавское шоссе', 'https://www.cian.ru/sale/suburban/263158735/'] 5
['cottage', '17999000', '99994', '180', '5.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/259693416/'] 6
['cottage', '18000000', '60000', '300', '7.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/253011806/'] 7
['house', '18000000', '75000', '240', '7.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/262957998/'] 8
['house', '18000000', '119205', 

['house', '20000000', '70423', '284', '12.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/259617881/'] 69
['house', '20000000', '80000', '250', '6.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/265324297/'] 70
['house', '20000000', '38314', '522', '10.0', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/256834622/'] 71
['house', '20150000', '134333', '150', '9.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/257738786/'] 72
['cottage', '20000000', '95238', '210', '15.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/264900134/'] 73
['house', '20000000', '425532', '47', '11.0', '1', 'Симферопольское шоссе', 'https://www.cian.ru/sale/suburban/257006299/'] 74
['house', '20300000', '156154', '130', '6.0', '2', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/260291361/'] 75
['house', '20500000', '136667', '150', '27.5', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264364596/'] 76
['house', '20500000', '941

['cottage', '24000000', '78689', '305', '5.0', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/263863207/'] 137
['house', '24000000', '84211', '285', '15.0', '1', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/261080400/'] 138
['cottage', '24000000', '80000', '300', '16.0', '2', 'Симферопольское шоссе', 'https://www.cian.ru/sale/suburban/258328178/'] 139
['house', '24120000', '102638', '235', '14.0', '2', 'Симферопольское шоссе', 'https://www.cian.ru/sale/suburban/257773114/'] 140
['cottage', '24000000', '111628', '215', '8.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/263077327/'] 141
['cottage', '24000000', '136364', '176', '6.43', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/262544311/'] 142
['house', '24000000', '133333', '180', '6.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/256985486/'] 143
['cottage', '24000000', '87273', '275', '18.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264508277/'] 144
['co

In [20]:
list10 = parse(links[9]) # Десятый запуск функции

<ipython-input-9-b26b98531458>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path='C:/Users/Андрей М Захаров/Desktop/Проект/geckodriver.exe')


['cottage', '24482000', '123959', '197.5', '12.0', '2', None, 'https://www.cian.ru/sale/suburban/260406967/'] 1
['cottage', '24800000', '55111', '450', '15.0', '4', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/235651465/'] 2
['house', '24850000', '90364', '275', '12.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/192945881/'] 3
['cottage', '25000000', '89190', '280.3', '15.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/251907456/'] 4
['cottage', '25000000', '67568', '370', '12.0', '3', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/239370720/'] 5
['cottage', '25000000', '49116', '509', '19.0', '2', 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/259365668/'] 6
['cottage', '24900000', '71143', '350', '12.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/229967969/'] 7
['house', '25000000', '65789', '380', '16.0', None, 'Киевское шоссе', 'https://www.cian.ru/sale/suburban/225885974/'] 8
['house', '25000000', '119048', '210',

['house', '27000000', '266798', '101.2', '5.0', '2', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/264977624/'] 69
['cottage', '27500000', '68750', '400', '12.0', '2', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/223990031/'] 70
['house', '27500000', '152778', '180', '10.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/264326374/'] 71
['cottage', '27500000', '229167', '120', '19.03', '1', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/259406394/'] 72
['cottage', '27500000', '56584', '486', '15.0', '3', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/215471255/'] 73
['house', '28000000', '92409', '303', '8.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/226477099/'] 74
['house', '28000000', '155556', '180', '12.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/239414199/'] 75
['cottage', '28000000', '82353', '340', '12.0', None, 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/256136796/'] 76
['cottage', '27900

['cottage', '31500000', '68330', '461', '16.0', '2', 'Минское шоссе', 'https://www.cian.ru/sale/suburban/223990030/'] 137
['house', '31990000', '139087', '230', '8.22', '2', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/248789465/'] 138
['cottage', '31900000', '145000', '220', '8.0', '3', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/257114875/'] 139
['house', '25000000', '69444', '360', '13.0', '3', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/263632965/'] 140
['cottage', '27500000', '56584', '486', '15.0', '3', 'Боровское шоссе', 'https://www.cian.ru/sale/suburban/215471255/'] 141
['cottage', '32990000', '143435', '230', '12.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/246257478/'] 142
['house', '28000000', '52045', '538', '24.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/256883077/'] 143
['house', '23990000', '99958', '240', '10.0', '2', 'Калужское шоссе', 'https://www.cian.ru/sale/suburban/251276213/'] 144
['house', '29900

In [21]:
s = 0

In [22]:
# Создаем функцию для записи полученных массивов в dataframe
def to_df(mass):
    global s
    for i in mass:
        df.loc[s] = i
        s += 1

In [23]:
# Применяем функцию к 10-ти полученным массивам
to_df(list1)
to_df(list2)
to_df(list3)
to_df(list4)
to_df(list5)
to_df(list6)
to_df(list7)
to_df(list8)
to_df(list9)
to_df(list10)
df

,type,price,prisePerEarthsquare,houseSquare,earthSquare,floors,place,URL
0,cottage,78000000,197468,395,22.3,4,Калужское шоссе,https://www.cian.ru/sale/suburban/248884179/
1,cottage,88500000,305172,290,17.58,2,Киевское шоссе,https://www.cian.ru/sale/suburban/256413509/
2,cottage,97007894,195976,495,20.0,2,Калужское шоссе,https://www.cian.ru/sale/suburban/238388603/
3,house,65655917,76755,855.4,20.1,2,Киевское шоссе,https://www.cian.ru/sale/suburban/249971632/
4,cottage,33500000,97101,345,10.0,2,Калужское шоссе,https://www.cian.ru/sale/suburban/264321673/
...,...,...,...,...,...,...,...,...
1476,house,23990000,99958,240,10.0,2,Калужское шоссе,https://www.cian.ru/sale/suburban/251276213/
1477,house,29900000,178188,167.8,13.0,None,Калужское шоссе,https://www.cian.ru/sale/suburban/235546341/
1478,house,33300000,100909,330,10.0,3,Калужское шоссе,https://www.cian.ru/sale/suburban/258318045/
1479,cottage,26500000,129268,205,9.0,2,Калужское шоссе,https://www.cian.ru/sale/suburban/258596533/


In [24]:
# Преобразуем dataframe в csv файл
df.to_csv(r"C:/Users/Андрей М Захаров/Desktop/Проект/dataset.csv", index=False, sep=";")